In [12]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from arb import Arbitrage
import time
from typing import List

In [13]:
CASAS_DE_APOSTA_BR = [
    'Dafabet',
    'Betfair',
    'Betway',
    'Betano',
    'Bet365',
    'Galera Bet',
    'Betsson',
    '22BET',
    'Betwinner',
    'Rabona',
    '888sport',
    'LeoVegas',
    'Rivalo',
    'Parimatch',
    'Sportaza',
    'Betkwiff',
    'Amuletobet',
    '20BET',
    'Powbet',
    'FezBet',
    'Campeonbet',
    'Melbet',
    'Pixbet',
    'BetWarrior',
    'Bet7',
    'Megapari',
    'KTO',
    'Vbet',
    'CampoBet',
    'CyberBet',
    'LVBET',
    'Betmaster',
    'Bodog',
    'Fansbet',
    'Betmotion'
]

SUBURL = '/soccer/brazil/serie-a/'
CASAS_DE_APOSTA_BR = list(map(lambda s: s.lower(), CASAS_DE_APOSTA_BR))

available_games = list()
sites = dict()

In [14]:
# Baixa as casas de aposta brasileiras para um determinado jogo
def retrieve_bet_houses_info_from_spec_game(parsed_text: List[str]):
    global sites
    inc = 0
    try:
        while True:
            casa_aposta, odd_a, empate, odd_b, payout = parsed_text[5+inc:10+inc]
            casa_aposta = casa_aposta.lower()
            if 'Log in to display the odds!' in [casa_aposta, odd_a, empate, odd_b, payout]:
                break

            if casa_aposta in CASAS_DE_APOSTA_BR:
                sites[casa_aposta] = {
                    "Odd_a":    float(odd_a),
                    "Empate":   float(empate),
                    "Odd_b":    float(odd_b),
                    "Payout":   payout
                }
            inc += 5
    except:
        pass
    
    # print("Sites obtidos: ", sites)
    return pd.DataFrame(sites).T

In [15]:
def retrieve_game_parsed_text(driver, URL: str):
    webElement = None
    
    print("URL atual: {}".format(URL))
    try:
        driver.get(URL)
    except:
        pass

    webElement = None
    while webElement is None:
        try:
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "odds-data-table")))
            webElement = driver.execute_script('return document.getElementById("odds-data-table").outerHTML')
        except TimeoutException:
            pass
    title = driver.execute_script('return document.title')
    print(title)
    
    assert isinstance(webElement, str)
    
    SoupElement = BeautifulSoup(webElement, 'html.parser')
    raw_text = [text for text in SoupElement.stripped_strings]
    # print(raw_text)

    return retrieve_bet_houses_info_from_spec_game(raw_text)

In [11]:
url = "https://www.oddsportal.com/soccer/brazil/serie-a"
driver = webdriver.Chrome()
webElement = None

try:
    driver.maximize_window()
    driver.get(url)
    # Login no oddsPortal
    time.sleep(2)
    driver.execute_script('document.getElementsByClassName("inline-btn-2 r button-dark")[0].click()')
    time.sleep(3)
    driver.find_element(By.ID, "login-username1").send_keys("picaxota2")
    driver.find_element(By.ID, "login-password1").send_keys("123456")
    driver.execute_script("document.getElementsByClassName('inline-btn-2')[2].click()")

    try:
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "tournamentTable")))
        webElement = driver.execute_script('return document.getElementById("tournamentTable").outerHTML')
    except TimeoutException:
        print("A tabela não foi carregada a tempo.")
        # driver.close()
        exit(-1)

    print("Jogos baixados com sucesso.")

except Exception as e:
    print(f"Um erro ocorreu ao executar o arquivo.")
    # driver.close()
    exit(-1)

try:
    assert isinstance(webElement, str)
except AssertionError:
    print("A tabela dos jogos não foi baixada corretamente.")
    # driver.close()
    exit(-1)

SoupElement = BeautifulSoup(webElement, 'html.parser')

# Obtém todos os jogos disponíveis na tabela
for href in SoupElement.find_all('a', href=True):
    if href['href'][:23]  == '/soccer/brazil/serie-a/' and href['href'] != SUBURL:
        available_games.append(href['href'])

for i, link in enumerate(available_games):
    df = retrieve_game_parsed_text(driver, "https://www.oddsportal.com" + link)
    print(df)
    print(Arbitrage(
        odd_a = df["Odd_a"].max(),
        odd_b = df["Odd_b"].max(),
        odd_draw = df["Empate"].max(),
        platform= str(),
    ).calculate_bets(verbose=True))

Jogos baixados com sucesso.
URL atual: https://www.oddsportal.com/soccer/brazil/serie-a/athletico-pr-internacional-IJ7Ocr3I/
Athletico-PR - Internacional Betting Odds, Soccer - Serie A
          Odd_a Empate Odd_b Payout
888sport   2.25    3.2  3.15  93.1%
bet365     2.37    3.2   3.2  95.5%
betfair     2.3    3.1  3.25  93.9%
betsson     2.3    3.3   3.1  94.3%
betway      2.3    3.2   3.1  93.5%
dafabet    2.31   3.15   3.0  92.3%
parimatch  2.35   3.25  3.15  95.2%
Total implied probability: 103.27%
----------------------------------
Profit margin: -3.27%
Expected profit: R$ -0.00
----------------------------------
Bet on outcome A: R$ 0.00
Bet on outcome B: R$ 0.00
Bet on draw: R$ 0.00
Can't guarantee profit with total implied probability greater than 100%
******************************************************************************
******************************************************************************
  Odd a  Odd Draw  Odd b  Bet a  Draw  Bet b  Profit Rate  Expected Pro